## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Saint-Philippe,RE,-21.3585,55.7679,78.80,69,0,3.44,light rain
1,1,Rikitea,PF,-23.1203,-134.9692,78.73,72,0,10.02,light rain
2,2,Tiksi,RU,71.6872,128.8694,-14.69,98,41,5.66,scattered clouds
3,3,Kaitangata,NZ,-46.2817,169.8464,58.75,74,61,3.71,broken clouds
4,4,Cape Town,ZA,-33.9258,18.4232,64.00,88,0,10.36,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Saint-Philippe,RE,-21.3585,55.7679,78.80,69,0,3.44,light rain
1,1,Rikitea,PF,-23.1203,-134.9692,78.73,72,0,10.02,light rain
5,5,Arraial Do Cabo,BR,-22.9661,-42.0278,78.80,65,0,17.27,clear sky
8,8,Dwarka,IN,22.2394,68.9678,79.32,73,0,12.68,clear sky
10,10,Atuona,PF,-9.8000,-139.0333,80.47,72,73,16.96,broken clouds
11,11,Jamestown,US,42.0970,-79.2353,75.20,27,1,12.66,clear sky
12,12,Bosaso,SO,11.2842,49.1816,81.27,52,0,7.27,clear sky
20,20,Butaritari,KI,3.0707,172.7902,81.52,78,99,12.01,light rain
21,21,Jertih,MY,5.7336,102.4897,78.80,94,20,2.30,few clouds
26,26,Makakilo City,US,21.3469,-158.0858,78.80,47,1,3.44,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                238
City                   238
Country                236
Lat                    238
Lng                    238
Max Temp               238
Humidity               238
Cloudiness             238
Wind Speed             238
Current Description    238
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                236
City                   236
Country                236
Lat                    236
Lng                    236
Max Temp               236
Humidity               236
Cloudiness             236
Wind Speed             236
Current Description    236
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Saint-Philippe,RE,78.80,light rain,-21.3585,55.7679,
1,Rikitea,PF,78.73,light rain,-23.1203,-134.9692,
5,Arraial Do Cabo,BR,78.80,clear sky,-22.9661,-42.0278,
8,Dwarka,IN,79.32,clear sky,22.2394,68.9678,
10,Atuona,PF,80.47,broken clouds,-9.8000,-139.0333,
11,Jamestown,US,75.20,clear sky,42.0970,-79.2353,
12,Bosaso,SO,81.27,clear sky,11.2842,49.1816,
20,Butaritari,KI,81.52,light rain,3.0707,172.7902,
21,Jertih,MY,78.80,few clouds,5.7336,102.4897,
26,Makakilo City,US,78.80,clear sky,21.3469,-158.0858,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df["Hotel Name"].replace("", np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Saint-Philippe,RE,78.80,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
1,Rikitea,PF,78.73,light rain,-23.1203,-134.9692,Pension Maro'i
5,Arraial Do Cabo,BR,78.80,clear sky,-22.9661,-42.0278,Pousada Porto Praia
8,Dwarka,IN,79.32,clear sky,22.2394,68.9678,Hotel Narayan Avezika Comfort
10,Atuona,PF,80.47,broken clouds,-9.8000,-139.0333,Villa Enata
11,Jamestown,US,75.20,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
12,Bosaso,SO,81.27,clear sky,11.2842,49.1816,Red Sea Hotel
20,Butaritari,KI,81.52,light rain,3.0707,172.7902,Isles Sunset Lodge
21,Jertih,MY,78.80,few clouds,5.7336,102.4897,One Utama Hotel
26,Makakilo City,US,78.80,clear sky,21.3469,-158.0858,Marriott's Ko Olina Beach Club


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))